# Train four different ML models with PyTorch to test linearity and independence

1. Train four different ML algorithums on all CMIP6 Global Climate Model large ensembles (LEs) with >15 members
2. Train four different ML algorithums on the multi-model CMIP6 ensemble (n=41) with 1st/2nd/3rd members as training/validation/test members
3. Remove one variable at a time from the linear model, trained on the first 75% of the LE members
4. Remove one variable at a time from the linear model, trained on the CMIP6 1st members  





In [2]:
import torch
import torch.nn as nn
import numpy as np
import scipy.stats as stats
import xarray as xr
import matplotlib.pyplot as plt
import datetime
import dask
import glob
import os
import re
import pickle
print(datetime.datetime.now())

2023-02-13 11:02:37.916563


In [3]:
torch.manual_seed(0); #for reproducability

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Load data and define useful functions and constants

In [23]:
#load useful data such as the GCM names and their DOIs
CMIP6_info = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
    +'CMIP6_modeling_center_members_doi.nc'
)

#loop through all GCMs and make list of all realizations common to regional
#SIC data and all CVDP variables
good_GCM_mem = {}
for GCM in CMIP6_info['model'].values:
    try:
        #open the regional SIC file and list the members, all members in file have
        #already gone through quality control to check no nan or 0 values
        SIC = xr.open_dataset(
            '/glade/work/cwpowell/low-frequency-variability/input_data/'\
            +f'Regional_SIC_detrended_lowpass_filter_{GCM}_1920_2014.nc')
        SIC_mems = SIC['member'].values

        #open the CVDP data and list all of the members which do not have nan values
        #for the AMO - if they do have nan values do not use list that member.
        CVDP = xr.open_dataset(
            '/glade/work/cwpowell/low-frequency-variability/input_data/'\
            +f'CVDP_standardized_linear_detrended_1920_2014_historical_{GCM}.nc')
        CVDP_mems = CVDP['member'].where(
            ~xr.ufuncs.isnan(CVDP['AMO']).max('time'), drop=True)

        #now list the members with both good SIC and CVDP data, if at least 1
        if len(np.intersect1d(CVDP_mems, SIC_mems)) > 0:
            good_GCM_mem[GCM] = np.sort(np.intersect1d(CVDP_mems, SIC_mems))
        else:
            print(GCM, 'No members with good SIC and CVDP data')
    
    except FileNotFoundError:
        print(GCM, 'Either SIC or CVDP data missing')
    
#remove CESM2 from the dictionary as replace with CESM2-LENS data
good_GCM_mem.pop('CESM2', None);

#save the member data
# with open('/glade/work/cwpowell/low-frequency-variability/raw_data/CMIP6_info/'\
#           +'CMIP6_members_CVDP_and_SIC.pickle', 'wb') as handle:
#     pickle.dump(good_GCM_mem, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
#define the train/validation/test split for the large ensembles with at least 15
#members
train_valid_test = [0.75, 0.15, 0.10]

LE_train_mem = {}
LE_valid_mem = {}
LE_test_mem  = {}
LE_GCM_list = []
for GCM in np.sort(list(good_GCM_mem.keys())):
    
    n_mem = len(good_GCM_mem[GCM])
    
    if n_mem > 15:
        LE_GCM_list.append(GCM)
        train_n = int(np.ceil(n_mem*train_valid_test[0]))
        test_n  = int(np.floor(n_mem*train_valid_test[2]))

        LE_train_mem[GCM] = good_GCM_mem[GCM][:train_n]
        LE_valid_mem[GCM] = good_GCM_mem[GCM][train_n:-test_n]
        LE_test_mem[GCM]  = good_GCM_mem[GCM][-test_n:]
        
LE_GCM_list = np.sort(LE_GCM_list)

#add in the CMIP6 member number codes
CVDP_CMIP6_xr = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/input_data/'\
    +'CVDP_standardized_linear_detrended_1920_2014_historical_CMIP6.nc')

SIC_CMIP6_xr = xr.open_dataset(
    '/glade/work/cwpowell/low-frequency-variability/input_data/'\
    +'Regional_SIC_detrended_lowpass_filter_CMIP6_1920_2014.nc')

LE_train_mem['CMIP6'] = CVDP_CMIP6_xr['member'].sel(
    member=slice(1000,1999)).values
LE_valid_mem['CMIP6'] = CVDP_CMIP6_xr['member'].sel(
    member=slice(2000,2999)).values
LE_test_mem['CMIP6']  = CVDP_CMIP6_xr['member'].sel(
    member=slice(10000,None)).values

In [6]:
#generate a list of variable and season names
CVDP_sample = xr.open_dataset('/glade/work/cwpowell/low-frequency-variability/'\
    +'input_data/CVDP_standardized_linear_detrended_1920_'\
    +'2014_historical_CanESM5.nc')

var_month_list = []
for i in CVDP_sample.to_array()['variable'].drop_sel(
    variable=['AMOC','NINO12','NINO3','NINO4']).values:
    for month_num in [1,4,7,10]:
        var_month_list.append(str(i)+'_'+str(month_num))
        
var_month_list.append('RAND_1')
var_month_list.append('RAND_4')
var_month_list.append('RAND_7')
var_month_list.append('RAND_10')

# Define functions for loading feature and target data, as well as training the 4 ML models

In [7]:
def load_CVDP(model_name, month_, lag_, start_end_yr, extra_drop=None, 
              white_noise=None):
    '''
    Load the training, validation and testing data of the climate modes of
    varaibility (features), corresponding to the time period of the sea ice
    data (target) and the lag time. Additionally, remove certain climate modes
    and/or include a white noise variable.
    
    Parameters
    ----------
    model_name: str,
        The name of the GCM which provides a sufficiently large ensemble.
    month_: int,
        The month of the target sea ice concentration, this will determine
        how many years lagged each season is, e.g. if sea ice concentration
        is for October the seasonal CVDP data will be lagged 2 years for DJF,
        MAM and JJA, but 3 years for SON.
    lag_: int,
        The number of years the CVDP data is offset before the sea ice data 
    start_end_yr: list, length 2 with integers,
        The start and end years (insclusive) for the sea ice concentration data.
    extra_drop: none or list of strings,
        If False, no additonal variables are dropped. If a list of a string or
        strings, those variables listed will be removed from the CVDP data. 
    white_noise: bool,
        If True, add a variable of normalized random values in a gaussian 
        distribution for the 4 seasons, all members and all years.
    
    Returns
    ----------
    CVDP_train: PyTorch tensor,
        The stacked CVDP for the training members, with shape 
        ([member x year],[variable x month]) e.g for the first 75% of the 65
        CanESM5 members used for training: ([49x74],[17x4]) = (3626,68).
    CVDP_valid: PyTorch tensor,
        The stacked CVDP for the validation members, with shape
        ([member x year],[variable x month]).
    CVDP_test: PyTorch tensor,
        The stacked CVDP for the testing members, with shape
        ([member x year],[variable x month]).    
    ''' 
    
    #load CVDP features and convert to seasonal data
    #select the NetCDF file with lowpass filtering or linear detrending
    CVDP_year_month = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/input_data/'\
        +'CVDP_standardized_linear_detrended_1920_2014_historical_'\
        +f'{model_name}.nc'
    )    

    CVDP_year_month = CVDP_year_month.to_array('variable').sortby('time')

    month_seperate = []
    for i in [1,4,7,10]:
        temp_data = CVDP_year_month.sel(
            time=CVDP_year_month['time.month']==i)
        temp_data['time'] = np.arange(1920,2015)
        month_seperate.append(temp_data)

    CVDP_data = xr.concat((month_seperate), dim='month')
    CVDP_data['month'] = [1,4,7,10]
    CVDP_data = CVDP_data.rename({'time':'year'})

    CVDP_data = CVDP_data.drop_sel(variable=['AMOC','NINO12','NINO3','NINO4'])
    
    if type(extra_drop) != type(None): #drop extra variables from the CVDP data
        CVDP_data = CVDP_data.drop_sel(variable=extra_drop)
        
    #now stack the CVDP data into X members and years, Y features
    CVDP_train = []
    CVDP_test  = []
    CVDP_valid = []
    for lag_season in [1,4,7,10]: 
        if lag_season >= month_:
            extra_year = 1
        else:
            extra_year = 0

        CVDP_month_data = CVDP_data.sortby('member')
            
        CVDP_month_data = CVDP_month_data.sel(
            month=lag_season).sel(
            year=slice(str(start_end_yr[0]-lag_-extra_year), 
                       str(start_end_yr[1]-lag_-extra_year)))
        CVDP_month_data['year'] = np.arange(
            0,start_end_yr[1]-start_end_yr[0]+1)
        
        #now, optionally add in white noise as 4 seasons of a new variable
        if white_noise:
            white_noise_month = (
                CVDP_month_data.copy().isel(variable=0) * 0 + np.random.normal(
                    loc=0, scale=1, size=(len(CVDP_month_data['member']),
                                          len(CVDP_month_data['year'])))
            )

            white_noise_month['variable'] = 'RAND'
            
            CVDP_month_data = xr.concat(
                (CVDP_month_data, white_noise_month), dim='variable'
            ) 
        
        CVDP_train.append(CVDP_month_data.sel(member=LE_train_mem[model_name]))
        CVDP_valid.append(CVDP_month_data.sel(member=LE_valid_mem[model_name]))
        CVDP_test.append(CVDP_month_data.sel(member=LE_test_mem[model_name]))
        
        
    CVDP_train_stacked = xr.concat((CVDP_train),'month').stack(
        member_time=('member','year')).stack(
        var_month=('variable','month'))
    CVDP_train = torch.Tensor(CVDP_train_stacked.values)
    
    CVDP_valid_stacked = xr.concat((CVDP_valid),'month').stack(
        member_time=('member','year')).stack(
        var_month=('variable','month'))
    CVDP_valid = torch.Tensor(CVDP_valid_stacked.values)

    CVDP_test_stacked = xr.concat((CVDP_test),'month').stack(
        member_time=('member','year')).stack(
        var_month=('variable','month'))
    CVDP_test = torch.Tensor(CVDP_test_stacked.values)
    
    return(CVDP_train, CVDP_valid, CVDP_test)

In [8]:
def load_SIC(model_name, month_, region_, start_end_yr):
    '''
    Load the sea ice concentration anomalies (targets) for a specific GCM,
    month and time period.
    
    Parameters
    ----------
    model_name: str,
        The name of the GCM which provides a sufficiently large ensemble.
    month_: int,
        The month of the target sea ice concentration anomalies. 
    region_: int,
        The region number for the sea ice concentration anomalies.
    start_end_yr: list, length 2 with integers,
        The start and end years (inclusive) for the sea ice concentration 
        anomalies.
    train_n: int,
        The highest member index to include in training data.
    test_n: int,
        The index of the lowest member index included in the test data.
    
    Returns
    ----------
    target_train: PyTorch tensor,
        The stacked sea ice concentration anomalies for the training members,
        with shape (member, year) e.g for the first 75% of the 65 CanESM5
        members used for training: (49,74). 
    target_valid: PyTorch tensor,
        The stacked sea ice concentration anomalies for the validation members,
        with shape (member, year).
    target_test: PyTorch tensor,
        The stacked sea ice concentration anomalies for the test members, with
        shape (member, year).
    
    '''
    #load SIC targets with the desired type of filtering/dretrending
    SIC_data = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/input_data/'\
        +f'Regional_SIC_detrended_lowpass_filter_{model_name}_1920_2014.nc'
    )
    
    #select the years for this analysis period and sort by member
    SIC_data = SIC_data['SIC'].sortby('member').sel(
        year=slice(str(start_end_yr[0]), str(start_end_yr[1])))
    
    #convert 2D xr.DataArray to 1D xr.DataArray to np.ndarray to torch.Tensor
    target_train = torch.from_numpy(
        SIC_data.sel(member=LE_train_mem[model_name]).sel(month=month_).sel(
            region=region_).stack(member_time=('member','year')).values
    )
    target_valid = torch.from_numpy(
        SIC_data.sel(member=LE_valid_mem[model_name]).sel(month=month_).sel(
            region=region_).stack(member_time=('member','year')).values
    )
    target_test = torch.from_numpy(
        SIC_data.sel(member=LE_test_mem[model_name]).sel(month=month_).sel(
            region=region_).stack(member_time=('member','year')).values
    )
    
    return(target_train, target_valid, target_test)

In [9]:
def train_4_ML_for_LE(model_name, month_, region_list, lag_list, start_end_yr, 
                      n_epoch, learn_rates, white_noise_=None):
    '''
    Train the 4 machine learning models for a given large ensemble on a 
    specificed of sea ice concentration data for a specified set of months,
    regions, and lags.
    
    Parameters
    ----------
    model_name: str,
        The name of the GCM which provides a sufficiently large ensemble.
    month_: int,
        The months of sea ice concentration anomalies on which to train the 
        machine learning model. 
    region_list: list of ints,
        A list of the regional sea ice anomalies to train the model on 
        separately.
    lag_list: list of ints,
        The range of lagged year on which to separately train the machine 
        learning model 
    start_end_yr: list, length 2 with integers,
        The start and end years (inclusive) for the sea ice concentration data.
    white_noise: bool,
        If True, add a variable of normalized random values in a gaussian 
        distribution for the 4 seasons, all members and all years.
    n_epoch: int, 
        number of epochs with which to train all of the machine learning 
        models.
    learn_rates: list of ints,
        List of 4 integers corresponding to the learning rate for each of the 4
        machine learning algorithums.                      
                      
    Returns
    ----------
    r_values_xr: xarray.Dataset,
        The pearson correlation coefficients for the 4 machine learning 
        algorithms from the validation data.
    all_1_grads_xr: xarray.Dataset,
        The gradients from the machine learning model using a simple multiple
        linear regression model.    
    '''

    if white_noise_:
        n_features = 15*4
    else:
        n_features = 14*4
        
    if model_name == 'CMIP6':
        doi_model = '10.5194/gmd-9-1937-2016'
    else:
        doi_model = CMIP6_info['doi'].sel(model=model_name).values
    
    all_1_weights = np.empty(
        [len(region_list), len(lag_list), n_features], dtype=float)

    all_r_values = np.empty(
        [len(region_list), len(lag_list), 4], dtype=float)

    for region_i, region_ in enumerate(region_list):

        for lag_i, lag_ in enumerate(lag_list):
            #load the feature and target data for the correct model, month,
            #region and lag
            CVDP_train, CVDP_valid, CVDP_test = load_CVDP(
                model_name, month_, lag_, start_end_yr, 
                extra_drop=None, white_noise=white_noise_
            )

            target_train, target_valid, target_test = load_SIC(
                model_name, month_, region_, start_end_yr, 
            )

            ML_model_computed = []
            for ML_i in range(4):
                if ML_i == 0:
                    ML_model_use = nn.Sequential(
                        nn.Linear(n_features,1,bias=False)
                    )
                elif ML_i == 1:
                    ML_model_use = nn.Sequential(
                        nn.Linear(n_features,1,bias=False), nn.ReLU()
                    )
                elif ML_i == 2:
                    ML_model_use = nn.Sequential(
                        nn.Linear(n_features,n_neurons), 
                        nn.Linear(n_neurons,n_neurons),
                        nn.Linear(n_neurons,1)
                    )
                elif ML_i == 3:
                    ML_model_use = nn.Sequential(
                        nn.Linear(n_features,n_neurons), nn.ReLU(),
                        nn.Linear(n_neurons,n_neurons), nn.ReLU(),
                        nn.Linear(n_neurons,1)
                    )

                optimizer = torch.optim.Adam(
                    params=ML_model_use.parameters(), 
                    lr=learn_rates[ML_i]
                )

                ##### train the model #####
                train_r, valid_r = [],[]
                for epoch in range(n_epoch):
                    # TRAIN
                    prediction = ML_model_use(CVDP_train)
                    optimizer.zero_grad() #reset the gradients to zeros
                    loss = loss_fcn(prediction[:,0].double(), target_train)
                    loss.backward()
                    optimizer.step()
                    train_r.append(
                        np.corrcoef(prediction[:,0].detach().numpy(),
                                    target_train.detach().numpy()
                                   )[1][0]
                    )

                    if ML_i == 0:
                        ML_model_computed.append(ML_model_use)

                    ##### validate the model #####
                    with torch.no_grad():
                        p_val = ML_model_use(CVDP_valid)
                        loss_val = loss_fcn(p_val[:,0].double(), 
                                            target_valid
                                           )

                        valid_r.append(
                            np.corrcoef(p_val[:,0].detach().numpy(),
                                        target_valid.detach().numpy()
                                       )[1][0]
                        )

                #select highest validation r value from all epochs
                all_r_values[region_i][lag_i][ML_i] = np.max(valid_r)

                if ML_i == 0: #record the weights of the best epoch
                    all_1_weights[region_i][lag_i] = np.ravel(
                        ML_model_computed[np.argmax(valid_r)][
                            0].weight[0,:].detach().numpy())

    if model_name == 'CMIP6':
        model_name = 'CMIP6 multi-model large ensemble'
    
    #save the r values to NetCDF every model and month
    r_values_xr = xr.Dataset(
        data_vars = {
            'r_value':(['region', 'lag', 'ML_model'], all_r_values)},
        coords = {
            'region':region_list, 'lag':lag_list, 'ML_model':range(4)},
    )

    r_values_xr_attrs = {
        'Description': 'Pearson correlation coefficient for validation '\
            +'data (15%) of the availible members of the global climate '\
            +f'model {model_name}. 4 different machine learning models '\
            +'fit the 16 features of seasonal climate modes computed by '\
            +'the Climate Variability Diagnostics Package with 2 year '\
            +'lowpass filtered sea ice concentration. These climate modes are '\
            +f'as follows: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, NPI, NAM, '\
            +'NPO, PNA, NAO, SAM, TAS. The ML models are '\
            +'trained at different lag times of 1-20 years and for each '\
            +'region (regions are defined by NSIDC MASIE-NH Version 1 '\
            +'(doi:10.7265/N5GT5K3K). The ML_model dimension refers to '\
            +'the 4 different ML architectures with PyTorch all using L1 '\
            +f'loss function and Adam optimizer and {n_epoch} '\
            +'epochs:'+'\n'+'1 - nn.Sequential'\
            +f'(nn.Linear({n_features},1,bias=False)), learning rate = '\
            +f'{learn_rates[0]}.'+'\n'+'2 - nn.Sequential(nn.Linear('\
            +f'{n_features},1,bias=False), nn.ReLU()), learning rate = '\
            +f'{learn_rates[1]}.'+'\n3 - nn.Sequential(nn.Linear('\
            +f'{n_features},{n_neurons}), nn.Linear({n_neurons},'\
            +f'{n_neurons}), nn.Linear({n_neurons},1)), learning rate = '\
            +f'{learn_rates[2]}.'+'\n'+'4 - nn.Sequential(nn.Linear('\
            +f'{n_features},{n_neurons}), nn.ReLU(),nn.Linear({n_neurons}'\
            +f',{n_neurons}), nn.ReLU(), nn.Linear({n_neurons},1)), '\
            +f'learning rate {learn_rates[3]}.',
        'Timestamp'  : str(datetime.datetime.utcnow().strftime(
            "%H:%M UTC %a %Y-%m-%d")),
        'Data source': f'CMIP6 global climate model {model_name}, '\
            +f'doi:{doi_model} sea ice concentration and climate modes '\
            +'calculated by the Climate Variability Diagnostics Package '\
            +'(doi:10.1002/2014EO490002)',
        'Analysis'   : 'https://github.com/chrisrwp/low-frequency-'\
            +'variability/blob/main/neural_network/4_PyTorch_model_'\
            +'configurations.ipynb',
    }

    r_values_xr.attrs = r_values_xr_attrs

    #save the linear 1 layer neural network weights to NetCDF
    if white_noise_:
        var_month_use = np.array(var_month_list).copy()
    else:
        var_month_use = np.array(var_month_list[:-4]).copy()
        
    all_1_weights_xr = xr.Dataset(
        data_vars = {
            'weights':(['region', 'lag', 'mode_month'], all_1_weights),
        },
        coords = {'region':region_list, 'lag':lag_list,
                  'mode_month':var_month_use,
        },
    )

    all_1_weights_xr_attrs = r_values_xr_attrs.copy()
    all_1_weights_xr_attrs['Description'] = 'Weights of the '\
        +'linear model fit for the validation data (15%) of the availible '\
        +f'members of the global climate model {model_name}. {n_features} '\
        +'features of seasonal climate modes computed by the Climate '\
        +'Variability Diagnostics Package with 2 year lowpass filtered sea '\
        +'ice concentration. These climate modes are as follows: AMO, IPO, '\
        +'NINO34, PDO, AMM, ATN, IOD, NPI, NAM, NPO, PNA, NAO, SAM, TAS. '\
        +'The model is trained at different lag times '\
        +'of 1-20 years and for each region (regions are defined by NSIDC '\
        +'MASIE-NH Version 1 (doi:10.7265/N5GT5K3K). The model uses '\
        +f'PyTorch with a L1 loss function, {n_epoch} epochs, Adam'\
        +f' optimizer and is defined by nn.Linear({n_features},1,'\
        +f'bias=False)), learning rate = {learn_rates[0]}.'

    all_1_weights_xr.attrs = all_1_weights_xr_attrs
        
    return(r_values_xr, all_1_weights_xr)

# 1. Train the 4 ML models with the first 75% of LE members, tested on next 15%

In [226]:
loss_fcn  = torch.nn.L1Loss() #keep all models sparse
n_neurons = 8

for month__ in np.arange(11,13):
    print(datetime.datetime.now(), month__)

    for model_name in LE_GCM_list:
        print(datetime.datetime.now(), model_name)

        r_values_xr, all_1_weights_xr = train_4_ML_for_LE(
            model_name, month_=month__, region_list=[1,2,3,4,5,6,11], 
            lag_list=np.arange(1,21), start_end_yr=[1941,2014], n_epoch=2000, 
            learn_rates=[5e-4, 5e-4, 1e-4, 2e-4], white_noise_=False,
        )    

        r_values_xr.to_netcdf(
            '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
            +f'validation_r_values_4ML_{model_name}_month_'\
            +f'{str(month__).zfill(2)}_var_14.nc')
        all_1_weights_xr.to_netcdf(
            '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
            +f'weights_linear_{model_name}_month_{str(month__).zfill(2)}_'\
            +'var_14.nc')
    

# 2. Train the 4 ML models with all CMIP6 GCMs, 1st/2nd/3rd+ members training/validation/testing

### Make the CMIP6 CVDP and SIC files as if 'CMIP6' was a GCM name and each GCM member 1 was a different member 

In [80]:
#obtain all of the train/validate/test GCM members
CMIP6_GCM_list = []
for GCM in np.sort(list(good_GCM_mem.keys())):    
    n_mem = len(good_GCM_mem[GCM])
    if n_mem > 2:
        CMIP6_GCM_list.append(GCM)

In [81]:
#gather all of the members together and save to NetCDF
CVDP_CMIP6 = []
SIC_CMIP6 = []

train_mem_i = 1000
valid_mem_i = 2000
test_mem_i  = 10000

for GCM in CMIP6_GCM_list:
    #loop through and append the correct members to the train, validation 
    #and testing groups with the 
    CVDP_data = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/input_data/'\
        +'CVDP_standardized_linear_detrended_1920_2014_historical_'\
        +f'{GCM}.nc'
    )
    CVDP_data = CVDP_data.sel(member=good_GCM_mem[GCM])
    
    new_mem_list = np.arange(test_mem_i, test_mem_i+len(CVDP_data['member'])-2)
    new_mem_list = np.insert(new_mem_list, 0, [train_mem_i, valid_mem_i])
    
    CVDP_data['member'] = new_mem_list
    CVDP_CMIP6.append(CVDP_data)
    
    #now do the same for the SIC data
    SIC_data = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/input_data/'\
        +f'Regional_SIC_detrended_lowpass_filter_{GCM}_1920_2014.nc'
    )
    SIC_data = SIC_data.sel(member=good_GCM_mem[GCM])
    
    SIC_data['member'] = new_mem_list   
    SIC_CMIP6.append(SIC_data)
    
    #now increase the initial value of the training, validation, and test member
    #element numbers
    train_mem_i += 1
    valid_mem_i += 1
    test_mem_i = test_mem_i + 1000

In [10]:
#save this CMIP6 data to NetCDF and include metadata
CVDP_CMIP6_xr = xr.concat((CVDP_CMIP6),dim='member').sortby('member')

CVDP_CMIP6_xr.attrs = {
    'Description' : 'Linearly detrended and standardized variables from the '\
        +'CVDP (Climate Variability Diagnostics Package) for all CMIP6 '\
        +'global climate models with at least 3 available members. Seasonal '\
        +'data for 1920-2014. The members can be decoded as follows: '\
        +'1000-1999 are the training members, 2000-2999 are the validation '\
        +'members, and 10000+ are the test members. The GCM is encoded as '\
        +'the last 2 digits for the training and validation members, and the '\
        +'first 2 digits +10 for the testing members. Note there is always '\
        +'1 member from each GCM for training and validation, and all '\
        +'remaining members are with the testing dataset. The GCM numbering '\
        +'referrs to the following: 0 ACCESS-CM2, 1 ACCESS-ESM1-5, '\
        +'2 BCC-CSM2-MR, 3 BCC-ESM1, 4 CAMS-CSM1-0, 5 CESM2-FV2, '\
        +'6 CESM2-LENS, 7 CESM2-WACCM, 8 CESM2-WACCM-FV2, 9 CIESM, '\
        +'10 CMCC-CM2-SR5, 11 CNRM-CM6-1, 12 CNRM-ESM2-1, 13 CanESM5, '\
        +'14 CanESM5-CanOE, 15 E3SM-1-0, 16 EC-Earth3, 17 EC-Earth3-CC, '\
        +'18 EC-Earth3-Veg, 19 EC-Earth3-Veg-LR, 20 FIO-ESM-2-0, '\
        +'21 GFDL-ESM4, 22 GISS-E2-1-G, 23 GISS-E2-1-H, 24 GISS-E2-2-G, '\
        +'25 GISS-E2-2-H, 26 HadGEM3-GC31-LL, 27 HadGEM3-GC31-MM, '\
        +'28 INM-CM5-0, 29 IPSL-CM6A-LR, 30 MIROC-ES2H, 31 MIROC-ES2L, '\
        +'32 MIROC6, 33 MPI-ESM-1-2-HAM, 34 MPI-ESM1-2-HR, 35 MPI-ESM1-2-LR, '\
        +'36 MRI-ESM2-0, 37 NESM3, 38 NorCPM1, 39 NorESM2-LM, 40 NorESM2-MM, '\
        +'41 UKESM1-0-LL. All members are sorted alphabetically before '\
        +'being divided into the three groups',
    'Units' :'standardized values',
    'Timestamp' : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Data source': 'CMIP6 historical simulations, computed by CVDP '\
        +'doi: 10.1002/2014EO490002.',
    'Analysis'   : 'https://github.com/chrisrwp/low-fequency-variability/'\
            +'neural_network/Train_4_ML_Models.ipynb',
}
    
CVDP_CMIP6_xr.to_netcdf(
    '/glade/work/cwpowell/low-frequency-variability/input_data/'\
    +'CVDP_standardized_linear_detrended_1920_2014_historical_CMIP6.nc')

SIC_CMIP6_xr = xr.concat((SIC_CMIP6),dim='member').sortby('member')
SIC_CMIP6_xr.attrs = {
    'Description' : '2 year lowpass filter of linearly detrended regional '\
        +'average sea ice concentration (SIC) in % for the climate model for '\
        +'all CMIP6 global climate models with at least 3 available members. '\
        +'Seasonal data for 1920-2014. The members can be decoded as follows: '\
        +'1000-1999 are the training members, 2000-2999 are the validation '\
        +'members, and 10000+ are the test members. The GCM is encoded as '\
        +'the last 2 digits for the training and validation members, and the '\
        +'first 2 digits +10 for the testing members. Note there is always '\
        +'1 member from each GCM for training and validation, and all '\
        +'remaining members are with the testing dataset. The GCM numbering '\
        +'referrs to the following: 0 ACCESS-CM2, 1 ACCESS-ESM1-5, '\
        +'2 BCC-CSM2-MR, 3 BCC-ESM1, 4 CAMS-CSM1-0, 5 CESM2-FV2, '\
        +'6 CESM2-LENS, 7 CESM2-WACCM, 8 CESM2-WACCM-FV2, 9 CIESM, '\
        +'10 CMCC-CM2-SR5, 11 CNRM-CM6-1, 12 CNRM-ESM2-1, 13 CanESM5, '\
        +'14 CanESM5-CanOE, 15 E3SM-1-0, 16 EC-Earth3, 17 EC-Earth3-CC, '\
        +'18 EC-Earth3-Veg, 19 EC-Earth3-Veg-LR, 20 FIO-ESM-2-0, '\
        +'21 GFDL-ESM4, 22 GISS-E2-1-G, 23 GISS-E2-1-H, 24 GISS-E2-2-G,'\
        +'25 GISS-E2-2-H, 26 HadGEM3-GC31-LL, 27 HadGEM3-GC31-MM, '\
        +'28 INM-CM5-0, 29 IPSL-CM6A-LR, 30 MIROC-ES2H, 31 MIROC-ES2L, '\
        +'32 MIROC6, 33 MPI-ESM-1-2-HAM, 34 MPI-ESM1-2-HR, 35 MPI-ESM1-2-LR, '\
        +'36 MRI-ESM2-0, 37 NESM3, 38 NorCPM1, 39 NorESM2-LM, 40 NorESM2-MM, '\
        +'41 UKESM1-0-LL. All members are sorted alphabetically before '\
        +'being divided into the three groups',
    'Timestamp' : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Data source': 'CMIP6 historical simulations',
    'Analysis'   : 'https://github.com/chrisrwp/low-fequency-variability/'\
            +'neural_network/Train_4_ML_Models.ipynb',
}
    
SIC_CMIP6_xr.to_netcdf(
    '/glade/work/cwpowell/low-frequency-variability/input_data/'\
    +'Regional_SIC_detrended_lowpass_filter_CMIP6_1920_2014.nc')

## Now train the 4 ML models on the CMIP6 data

In [22]:
loss_fcn  = torch.nn.L1Loss() #keep all models sparse
n_neurons = 8

for month__ in np.arange(1,13):
    print(datetime.datetime.now(), month__)

    r_values_xr, all_1_weights_xr = train_4_ML_for_LE(
        'CMIP6', month_=month__, region_list=[1,2,3,4,5,6,11], 
        lag_list=np.arange(1,21), start_end_yr=[1941,2014], n_epoch=2000, 
        learn_rates=[5e-4, 5e-4, 1e-4, 2e-4], white_noise_=False,
    )    

    r_values_xr.to_netcdf(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'validation_r_values_4ML_CMIP6_month_'\
        +f'{str(month__).zfill(2)}_var_14.nc')
    all_1_weights_xr.to_netcdf(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'weights_linear_CMIP6_month_{str(month__).zfill(2)}_'\
        +'var_14.nc')


## Test the CMIP6-trained linear model with 3rd members

In [19]:
#load weight data 
CMIP6_weights = []
for month_ in np.arange(1,13):
    all_1_weights_xr = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'weights_linear_CMIP6_month_{str(month__).zfill(2)}_'\
        +'var_14.nc'
    )
    CMIP6_weights.append(all_1_weights_xr['weights'])

CMIP6_weights = xr.concat((CMIP6_weights), dim='month')
CMIP6_weights['month'] = np.arange(1,13)

In [21]:
month_r_vals = []
for month_ in np.arange(1,13):
    print(datetime.datetime.now(), month_)
    lag_r_vals = []
    for lag_ in np.arange(1,21):

        CVDP_train, CVDP_valid, CVDP_test = load_CVDP(
            'CMIP6', month_, lag_, [1941,2014], 
            extra_drop=None, white_noise=False,
        )
                
        region_r_vals = []
        for region_ in [1,2,3,4,5,6,11]:
            target_train, target_valid, target_test = load_SIC(
                 'CMIP6', month_, region_, [1941,2014], 
            )
            
            prediction = np.sum(
                np.array(CVDP_test) 
                * np.tile(CMIP6_weights.sel(month=month_).sel(
                    region=region_).sel(lag=lag_), 
                [38850,1]), axis=1
            )
            
            all_test_mem = []
            for i in range(525):
                all_test_mem.append(
                    np.corrcoef(prediction[i*74:(i+1)*74], 
                                target_test[i*74:(i+1)*74])[0][1])
                
            region_r_vals.append(all_test_mem)
            
        lag_r_vals.append(region_r_vals)
        
    month_r_vals.append(lag_r_vals)
    
month_r_vals_xr = xr.Dataset(
    data_vars = {'r_value':(['month','lag','region','member'], month_r_vals)},
    coords = {'month':np.arange(1,13), 'lag':np.arange(1,21), 
              'region':[1,2,3,4,5,6,11], 
              'member':SIC_CMIP6_xr['member'][-525:].values,
             },
    )

In [127]:
#loop through the r values and group by GCM
r_val_by_GCM = []
for GCM_i, GCM in enumerate(CMIP6_GCM_list):
    begin_mem = str(10000+(GCM_i*1000))
    end_mem   = str(10999+(GCM_i*1000))
    
    data = month_r_vals_xr.sel(member=slice(begin_mem,end_mem))
    data['member'] = np.arange(0,len(data['member']))
    r_val_by_GCM.append(data)

r_val_by_GCM = xr.concat((r_val_by_GCM), dim='model_name')
r_val_by_GCM['model_name'] = CMIP6_GCM_list

r_val_by_GCM.attrs = {
    'Description': 'Pearson correlation coefficient for test data (3rd and '\
        +'later members of 41 global climate models as follows: '\
        +f'{CMIP6_GCM_list}. Model train and validated on the 1st and 2nd '\
        +'members of the same 41 GCMs. The model features were the following '\
        +'modes of variability: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, NPI, '\
        +'NAM, NPO, PNA, NAO, SAM, TAS. The targets were 2 year lowpass '\
        +'filtered regional Arctic sea ice concentration anomalies for lag '\
        +'times of 1-20 years. Regions are defined by NSIDC MASIE-NH Version'\
        +' 1 (doi:10.7265/N5GT5K3K). The climate modes are computed by the '\
        +'Climate Variability Diagnostics Package (CVDP). The model was '\
        +'trained using an L1 loss function and Adam optimizer with 2000 '\
        +'epochs: nn.Sequential(nn.Linear(56,1,bias=False)), with a '\
        +'learning rate of 5e-4.',
    'Timestamp'  : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Data source': f'CMIP6 global climate models for historical simulations '\
        +'with sea ice concentration output, climate modes calculated by CVDP '\
        +'(doi:10.1002/2014EO490002)',
    'Analysis'   : 'https://github.com/chrisrwp/low-frequency-'\
        +'variability/blob/main/neural_network/Train_4_ML_Models.ipynb',
}

r_val_by_GCM.to_netcdf(
    '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
    +f'test_r_values_linear_CMIP6_var_15_all_months_all_regions.nc'
)

# 3. Remove one variable at a time from the LE linear models, retrain on the useful variables

In [10]:
def train_linear_model_remove(model_name, month_, region_list, lag_list, 
                              start_end_yr, n_epoch, learn_rate, 
                              extra_drop_=None, white_noise_=None):
    '''
    Train the 4 machine learning models for a given large ensemble on a 
    specificed of sea ice concentration data for a specified set of months,
    regions, and lags.
    
    Parameters
    ----------
    model_name: str,
        The name of the GCM which provides a sufficiently large ensemble.
    month_: int,
        The months of sea ice concentration anomalies on which to train the 
        machine learning model. 
    region_list: list of ints,
        A list of the regional sea ice anomalies to train the model on 
        separately.
    lag_list: list of ints,
        The range of lagged year on which to separately train the machine 
        learning model 
    start_end_yr: list, length 2 with integers,
        The start and end years (inclusive) for the sea ice concentration data.
    extra_drop: none or list of strings,
        If False, no additonal variables are dropped. If a list of a string or
        strings, those variables listed will be removed from the CVDP data. 
    white_noise: bool,
        If True, add a variable of normalized random values in a gaussian 
        distribution for the 4 seasons, all members and all years.
    n_epoch: int, 
        number of epochs with which to train all of the machine learning 
        models.
    learn_rate: int,
        Integer corresponding to the learning rate.         
                      
    Returns
    ----------
    r_values_xr: xarray.Dataset,
        The pearson correlation coefficients for the linear model
        for the validation data.
    all_1_grads_xr: xarray.Dataset,
        The gradients from the machine learning model using a simple multiple
        linear regression model.    
    '''
    if white_noise_:
        fewer_var = 0
    else:
        fewer_var = 1
    
    if type(extra_drop_) != type(None):
        n_features = int((15-len(extra_drop_)-fewer_var)*4)
    else:
        n_features = int((15-fewer_var)*4)
    
    if model_name == 'CMIP6':
        doi_model = '10.5194/gmd-9-1937-2016'
    else:
        doi_model = CMIP6_info['doi'].sel(model=model_name).values
    
    all_1_weights = np.empty([len(region_list), len(lag_list), n_features], 
                             dtype=float)
    
    all_r_values = np.empty([len(region_list), len(lag_list)], dtype=float)

    for region_i, region_ in enumerate(region_list):
        
        for lag_i, lag_ in enumerate(lag_list):
            
            #load the feature and target data for the correct model, month,
            #region and lag
            CVDP_train, CVDP_valid, CVDP_test = load_CVDP(
                model_name, month_, lag_, start_end_yr, 
                extra_drop=extra_drop_, white_noise=white_noise_
            )
            
            target_train, target_valid, target_test = load_SIC(
                model_name, month_, region_, start_end_yr, 
            )

            ML_model_computed = []
            ML_model_use = nn.Sequential(nn.Linear(n_features,1,bias=False))

            optimizer = torch.optim.Adam(
                params=ML_model_use.parameters(), 
                lr=learn_rate
            )

            ##### train the model #####
            train_r, valid_r = [],[]
            for epoch in range(n_epoch):
                # TRAIN
                prediction = ML_model_use(CVDP_train)
                optimizer.zero_grad() #reset the gradients to zeros
                loss = loss_fcn(prediction[:,0].double(), target_train)
                loss.backward()
                optimizer.step()
                train_r.append(
                    np.corrcoef(prediction[:,0].detach().numpy(),
                                target_train.detach().numpy()
                               )[1][0]
                )

                ML_model_computed.append(ML_model_use)

                ##### validate the model #####
                with torch.no_grad():
                    p_val = ML_model_use(CVDP_valid)
                    loss_val = loss_fcn(p_val[:,0].double(), target_valid)
                    valid_r.append(np.corrcoef(p_val[:,0].detach().numpy(),
                                    target_valid.detach().numpy())[1][0])

                    #select highest validation r value from all epochs
                    all_r_values[region_i][lag_i] = np.max(valid_r)

                    all_1_weights[region_i][lag_i] = np.ravel(
                        ML_model_computed[np.argmax(valid_r)][                               
                            0].weight[0,:].detach().numpy())

    if model_name == 'CMIP6':
        model_name = 'CMIP6 multi-model large ensemble'
        
    if (white_noise_ == None) and (extra_drop_ == None):
        mode_month_list = [
            item for item in var_month_list if 'RAND' not in item]
    elif (len(extra_drop_) == 1) and (white_noise_ == True):
        mode_month_list = [
            item for item in var_month_list if extra_drop_[0] not in item]
    else:
        extra_drop_inc_rand = np.append(extra_drop_, 'RAND')
        mode_month_list = var_month_list.copy()
        for i in extra_drop_inc_rand:
            mode_month_list = [
                item for item in mode_month_list if i not in item]

    
    #save the r values to NetCDF every model and month
    r_values_xr = xr.Dataset(
        data_vars = {
            'r_value':(['region', 'lag'], all_r_values)},
        coords = {
            'region':region_list, 'lag':lag_list},
    )

    #save the linear 1 layer neural network weights to NetCDF
    all_1_weights_xr = xr.Dataset(
        data_vars = {
            'weights':(['region', 'lag', 'mode_month'], all_1_weights),
        },
        coords = {'region':region_list, 'lag':lag_list, 
                  'mode_month':mode_month_list,
        },
    )
        
    return(r_values_xr, all_1_weights_xr)

## Remove one variable at a time from the LE models

In [11]:
#use the following dictionaries to determine which months and regions to 
#retrain the model on. Based on best 5 year lag r2 above persistence. 

#use the months from the best CMIP6 values
good_GCM_months = {
    '1': [9, 9, 9, 9, 9, 9],
    '2': [8, 8, 8, 8, 8, 8, 8],
    '3': [10, 10, 10, 10],
    '4': [10, 10],
    '5': [10, 10],
    '6': [9, 9, 9],
    '11':[8, 8, 8]
}

good_GCM_names = {
    '1': ['CanESM5', 'CESM2-LENS', 'MIROC6', 'GISS-E2-1-G', 'IPSL-CM6A-LR',
          'GISS-E2-1-H'],
    '2': ['CanESM5', 'MIROC6', 'GISS-E2-1-G', 'ACCESS-ESM1-5', 'IPSL-CM6A-LR',
          'MIROC-ES2L', 'UKESM1-0-LL'],
    '3': ['CanESM5', 'MIROC6', 'IPSL-CM6A-LR', 'MIROC-ES2L'],
    '4': ['CanESM5', 'UKESM1-0-LL'],
    '5': ['CanESM5', 'UKESM1-0-LL'],
    '6': ['CanESM5', 'GISS-E2-1-H', 'UKESM1-0-LL'],
    '11':['CanESM5', 'ACCESS-ESM1-5', 'UKESM1-0-LL']
}


In [20]:
loss_fcn  = torch.nn.L1Loss() #keep all models sparse

var_list_use = list(CVDP_sample.to_array()['variable'].drop_sel(
    variable=['AMOC','NINO12','NINO3','NINO4']).values)+['RAND']


for region_key in [1,2,3,4,5,6,11]:
    print(datetime.datetime.now(), region_key)
    
    for GCM_i, GCM in enumerate(good_GCM_names[str(region_key)]):
        print(datetime.datetime.now(), GCM)
        
        doi_model = CMIP6_info['doi'].sel(model=GCM).values

        month__ = good_GCM_months[str(region_key)][GCM_i]
        
        #skip this one if already run this analysis
        if len(glob.glob(
            '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
            +f'validation_r_values_linear_{GCM}_month_{str(month__).zfill(2)}_'\
            +f'var_15_drop_1_region_{region_key}.nc')):
            continue

        r_values_list = []
        weights_list  = []
        for drop_var in var_list_use:

            if drop_var == 'RAND':
                r_values_xr, all_weights_xr = train_linear_model_remove(
                    model_name=GCM, month_=month__, region_list=[region_key], 
                    lag_list=np.arange(1,21), start_end_yr=[1941,2014], 
                    n_epoch=2000, learn_rate=5e-4, extra_drop_=None,
                    white_noise_=None,
                )

            else:
                r_values_xr, all_weights_xr = train_linear_model_remove(
                    model_name=GCM, month_=month__, region_list=[region_key], 
                    lag_list=np.arange(1,21), start_end_yr=[1941,2014], 
                    n_epoch=2000, learn_rate=5e-4, extra_drop_=[drop_var],
                    white_noise_=True,
                )  

            r_values_list.append(r_values_xr)
            weights_list.append(all_weights_xr)

        r_values_save = xr.concat((r_values_list), dim='drop_var')
        r_values_save['drop_var'] = var_list_use

        weights_save = xr.concat((weights_list), dim='drop_var')
        weights_save['drop_var'] = var_list_use

        r_values_attrs = {
            'Description': 'Pearson correlation coefficient for validation '\
                +f'data (15%) of members from the global climate model {GCM}. '\
                +'14 of the 15 modes '\
                +'of variability as computed by the Climate Variability '\
                +'Diagnostics Package (CVDP) with 2 year lowpass filtered '\
                +'sea ice concentration. These climate modes are as follows: '\
                +'AMO, IPO, NINO34, PDO, AMM, ATN, IOD, NPI, NAM, NPO, PNA, '\
                +'NAO, SAM, TAS, RAND. The linear model is trained at '\
                +'different lag times of 1-20 years and for each '\
                +'region (regions are defined by NSIDC MASIE-NH Version 1 '\
                +'(doi:10.7265/N5GT5K3K). Using an L1 loss function and Adam '\
                +'optimizer with 2000 epochs: nn.Sequential(nn.Linear(56,1,'\
                +'bias=False)), learning rate = 5e-4.',
            'Timestamp'  : str(datetime.datetime.utcnow().strftime(
                "%H:%M UTC %a %Y-%m-%d")),
            'Data source': f'CMIP6 global climate model {GCM}, doi:'\
                +f'{doi_model} for historical sea ice concentration '\
                +'simulation, climate modes calculated by CVDP '\
                +'(doi:10.1002/2014EO490002)',
            'Analysis'   : 'https://github.com/chrisrwp/low-frequency-'\
                +'variability/blob/main/neural_network/Train_4_ML_Models.ipynb',
        }

        r_values_save.attrs = r_values_attrs
        r_values_save.to_netcdf(
            '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
            +f'validation_r_values_linear_{GCM}_month_{str(month__).zfill(2)}_'\
            +f'var_15_drop_1_region_{region_key}.nc'
        )


        weights_attrs = r_values_attrs.copy()
        weights_attrs['Description'] = 'Linear weights including 14 of the '\
            +'15 modes of variability as computed by the Climate Variability '\
            +'Diagnostics Package (CVDP) with 2 year lowpass filtered sea ice '\
            +'concentration. The linear model is trained/validated on the '\
            +'first 75/15% of the members from the global climate model '\
            +f'{GCM}. The climate modes are as follows: AMO, IPO, NINO34, '\
            +'PDO, AMM, ATN, IOD, NPI, NAM, NPO, PNA, NAO, SAM, TAS, RAND. '\
            +'The linear model is trained at different lag times of 1-20 '\
            +'years and for each region (regions are defined by NSIDC '\
            +'MASIE-NH Version 1 (doi:10.7265/N5GT5K3K). Using an L1 loss '\
            +'function and Adam optimizer with 2000 epochs: '\
            +'nn.Sequential(nn.Linear(56,1,bias=False)), learning rate = 5e-4.'

        weights_save.attrs = weights_attrs
        weights_save.to_netcdf(
            '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
            +f'weights_linear_{GCM}_month_{str(month__).zfill(2)}_'\
            +f'var_15_drop_1_region_{region_key}.nc'
        )

### Remove all variables with a worse effect on skill than a random variable for the LEs

In [14]:
#load all of the drop1 LE data
r_vals_LE_drop1 = []

for region_i, region_ in enumerate([1,2,3,4,5,6,11]):
    
    region_r_vals = []
    for GCM_i, GCM in enumerate(good_GCM_names[str(region_)]):
        month_ = good_GCM_months[str(region_)][GCM_i]

        r_values = xr.open_dataset(
            '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
            +f'validation_r_values_linear_{GCM}_month_{str(month_).zfill(2)}'\
            +f'_var_15_drop_1_region_{region_}.nc'
        )
        
        region_r_vals.append(r_values['r_value'])
    
    region_r_vals = xr.concat((region_r_vals), dim='model_name')
    region_r_vals['model_name'] = good_GCM_names[str(region_)]    
    r_vals_LE_drop1.append(region_r_vals)
    
r_vals_LE_drop1 = xr.concat((r_vals_LE_drop1), dim='region')
r_vals_LE_drop1['region'] = [1,2,3,4,5,6,11]

In [18]:
loss_fcn  = torch.nn.L1Loss() #keep all models sparse
lag_range = np.arange(5,11)

month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 'November', 'December']

for region_key in [1,2,3,4,5,6,11]:
    print(datetime.datetime.now(), region_key)
    
    GCM_r_vals = []
    GCM_weights = []
    drop_dict = {}
    for GCM_i, GCM in enumerate(good_GCM_names[str(region_key)]):
        
        doi_model = CMIP6_info['doi'].sel(model=GCM).values

        month__ = good_GCM_months[str(region_key)][GCM_i]
        
        r_diff = r_vals_LE_drop1.sel(region=region_key).sel(lag=lag_range).mean(
            'lag').sel(model_name=GCM)**2 - r_vals_LE_drop1.sel(
            region=region_key).sel(lag=lag_range).mean('lag').sel(
            model_name=GCM).sel(drop_var='RAND')**2

        drop_list = r_diff['drop_var'].where(r_diff>0, drop=True).values
        drop_dict[GCM] = drop_list
        print(datetime.datetime.now(), GCM, drop_list)
        
        r_values_xr, all_weights_xr = train_linear_model_remove(
            model_name=GCM, month_=month__, region_list=[region_key], 
            lag_list=np.arange(1,21), start_end_yr=[1941,2014], 
            n_epoch=2000, learn_rate=5e-4, extra_drop_=drop_list,
            white_noise_=False,
        )  
        
        GCM_r_vals.append(r_values_xr)
        GCM_weights.append(all_weights_xr)
        
    region_r_vals = xr.concat((GCM_r_vals), dim='model_name')
    region_r_vals['model_name'] = good_GCM_names[str(region_key)]
    
    region_weights = xr.concat((GCM_weights), dim='model_name')
    region_weights['model_name'] = good_GCM_names[str(region_key)]

    r_values_attrs = {
        'Description': 'Pearson correlation coefficient for validation '\
            +'data (15%) of members from the global climate models as '\
            +f'follows {good_GCM_names[str(region_key)]} for '\
            +f'{month_names[month__-1]}, with modes of '\
            +f'variability yielding lower predictive skill than a random '\
            +f'variable dropped as follows: {drop_dict}. The full list of 15 '\
            +'variables is: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, NPI, NAM, '\
            +'NPO, PNA, NAO, SAM, TAS, as computed by the Climate Variability '\
            +'Diagnostics Package (CVDP) with 2 year lowpass filtered '\
            +'sea ice concentration. The linear model is trained at '\
            +f'different lag times of 1-20 years, the region is {region_key} '\
            +'as defined by NSIDC MASIE-NH Version 1 (doi:10.7265/N5GT5K3K). '\
            +'Training uses an L1 loss function and Adam optimizer with 2000 '\
            +'epochs: nn.Sequential(nn.Linear(n_features,1,bias=False)), '\
            +'learning rate = 5e-4.',
        'Timestamp'  : str(datetime.datetime.utcnow().strftime(
            "%H:%M UTC %a %Y-%m-%d")),
        'Data source': f'CMIP6 global climate model historical simulations '\
            +'for historical sea ice concentration, with climate modes '\
            +'calculated by CVDP (doi:10.1002/2014EO490002)',
        'Analysis'   : 'https://github.com/chrisrwp/low-frequency-'\
            +'variability/blob/main/neural_network/Train_4_ML_Models.ipynb',
    }

    region_r_vals.attrs = r_values_attrs
    region_r_vals.to_netcdf(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'validation_r_values_linear_LEs_region_{region_key}_month_{month__}'\
        +f'dropped_useless_vars.nc'
    )


    weights_attrs = r_values_attrs.copy()
    weights_attrs['Description'] = 'Linear weights of the useful modes of '\
        +'variability as computed by the Climate Variability Diagnoistics'\
        +'Package (CVDP) with 2 year lowpass filtered sea ice concentration '\
        +f'for {month_names[month__-1]}. The linear '\
        +f'model is trained/validated on the first 75/15% of the members '\
        +f'from the global climate models {good_GCM_names[str(region_key)]}. '\
        +'Climate modes which do not yield a higher predictive skill than a '\
        +f'random variable are removed as follows {drop_dict}. The full list '\
        +'of variables is as follows: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, '\
        +'NPI, NAM, NPO, PNA, NAO, SAM, TAS. The linear model is trained at '\
        +f'different lag times of 1-20 years and for region {region_key} as'\
        +'defined by NSIDC MASIE-NH Version 1 (doi:10.7265/N5GT5K3K). '\
        +'Training uses an L1 loss function and Adam optimizer with 2000 '\
        +'epochs: nn.Sequential(nn.Linear(n_features,1,bias=False)), '\
        +'learning rate = 5e-4.'

    region_weights.attrs = weights_attrs
    region_weights.to_netcdf(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'weights_linear_LEs_region_{region_key}_month_{month__}_'\
        +f'dropped_useless_vars.nc'
    )

# 4. Remove one variable at a time from the CMIP6 model, and retrain on the useful variables

In [19]:
loss_fcn  = torch.nn.L1Loss() #keep all models sparse

var_list_use = list(CVDP_sample.to_array()['variable'].drop_sel(
        variable=['AMOC','NINO12','NINO3','NINO4']).values)+['RAND']


for month__ in np.arange(1,13):
    print(datetime.datetime.now(), month__)
    r_values_list = []
    weights_list  = []
    
    for drop_var in var_list_use:
        print(datetime.datetime.now(), drop_var)
        
        if drop_var == 'RAND':
            r_values_xr, all_weights_xr = train_linear_model_remove(
                'CMIP6', month_=month__, region_list=[1,2,3,4,5,6,11], 
                lag_list=np.arange(1,21), start_end_yr=[1941,2014], 
                n_epoch=2000, learn_rate=5e-4, extra_drop_=None,
                white_noise_=None,
            )
            
        else:
            r_values_xr, all_weights_xr = train_linear_model_remove(
                'CMIP6', month_=month__, region_list=[1,2,3,4,5,6,11], 
                lag_list=np.arange(1,21), start_end_yr=[1941,2014], 
                n_epoch=2000, learn_rate=5e-4, extra_drop_=drop_var,
                white_noise_=True,
            )    

        r_values_list.append(r_values_xr)
        weights_list.append(all_weights_xr)
        
    r_values_save = xr.concat((r_values_list), dim='drop_var')
    r_values_save['drop_var'] = var_list_use
    
    weights_save = xr.concat((weights_list), dim='drop_var')
    weights_save['drop_var'] = var_list_use

    r_values_attrs = {
        'Description': 'Pearson correlation coefficient for validation '\
            +'data, 2nd member from 41 CMIP6 GCMs. Training data was the '\
            +'first member. 14 of the 15 modes of variability as computed by '\
            +'the Climate Variability Diagnostics Package (CVDP) with 2 year '\
            +'lowpass filtered sea ice concentration. These climate modes are '\
            +'as follows: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, NPI, NAM, '\
            +'NPO, PNA, NAO, SAM, TAS, RAND. The linear model is trained '\
            +'at different lag times of 1-20 years and for each '\
            +'region (regions are defined by NSIDC MASIE-NH Version 1 '\
            +'(doi:10.7265/N5GT5K3K). Using an L1 loss function and Adam '\
            +'optimizer with 2000 epochs: nn.Sequential(nn.Linear(56,1,'\
            +'bias=False)), learning rate = 5e-4.',
        'Timestamp'  : str(datetime.datetime.utcnow().strftime(
            "%H:%M UTC %a %Y-%m-%d")),
        'Data source': f'CMIP6 global climate model multi-model ensemble'\
            +f'doi:10.5194/gmd-9-1937-2016 for historical sea ice '\
            +'concentration simulation, climate modes calculated by CVDP '\
            +'(doi:10.1002/2014EO490002)',
        'Analysis'   : 'https://github.com/chrisrwp/low-frequency-'\
            +'variability/blob/main/neural_network/Train_4_ML_Models.ipynb',
    }

    r_values_save.attrs = r_values_attrs
    r_values_save.to_netcdf(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'validation_r_values_linear_CMIP6_month_{str(month__).zfill(2)}_'\
        +'var_15_drop_1.nc'
    )


    weights_attrs = r_values_attrs.copy()
    weights_attrs['Description'] = 'Linear weights including 14 of the 15 '\
        +'modes of variability as computed by the Climate Variability '\
        +'Diagnostics Package (CVDP) with 2 year lowpass filtered sea ice '\
        +'concentration. The linear model is trained on the 1st member of all '\
        +'CMIP6 GCMs and validated on the second members. The climate modes '\
        +'are as follows: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, NPI, NAM, '\
        +'NPO, PNA, NAO, SAM, TAS, RAND. The linear model is trained at '\
        +'different lag times of 1-20 years and for each region (regions are '\
        +'defined by NSIDC MASIE-NH Version 1 (doi:10.7265/N5GT5K3K). Using '\
        +'an L1 loss function and Adam optimizer with 2000 epochs: '\
        +'nn.Sequential(nn.Linear(56,1,bias=False)), learning rate = 5e-4.'

    weights_save.attrs = weights_attrs
    weights_save.to_netcdf(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'weights_linear_CMIP6_month_{str(month__).zfill(2)}_'\
        +'var_15_drop_1.nc'
    )

## Retrain the CMIP6 linear model with only the useful variables

In [28]:
#load all of the drop1 CMIP6 data
r_vals_CMIP6_drop1 = []
for month_ in np.arange(1,13):

    r_values = xr.open_dataset(
        '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
        +f'validation_r_values_linear_CMIP6_month_{str(month_).zfill(2)}'\
        +f'_var_15_drop_1.nc'
    )

    r_vals_CMIP6_drop1.append(r_values['r_value'])
    
r_vals_CMIP6_drop1 = xr.concat((r_vals_CMIP6_drop1), dim='month')
r_vals_CMIP6_drop1['month'] = np.arange(1,13)

In [37]:
#obtain all of the train/validate/test GCM members
CMIP6_GCM_list = []
for GCM in np.sort(list(good_GCM_mem.keys())):    
    n_mem = len(good_GCM_mem[GCM])
    if n_mem > 2:
        CMIP6_GCM_list.append(GCM)

In [42]:
loss_fcn  = torch.nn.L1Loss() #keep all models sparse
lag_range = np.arange(5,11)

month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 'November', 'December']

#select the months where multi-model ensemble has highest r2 above persistence
good_CMIP6_months = [9, 8, 10, 10, 10, 9, 8]

CMIP6_r_vals  = []
CMIP6_weights = []
for region_i, region_key in enumerate([1,2,3,4,5,6,11]):
    print(datetime.datetime.now(), region_key)

    month__ = good_CMIP6_months[region_i]

    r_diff = r_vals_CMIP6_drop1.sel(region=region_key).sel(month=month__).sel(
        lag=lag_range).mean('lag')**2 - r_vals_CMIP6_drop1.sel(
        region=region_key).sel(month=month__).sel(lag=lag_range).mean(
        'lag').sel(drop_var='RAND')**2

    drop_list = r_diff['drop_var'].where(r_diff>0, drop=True).values
    drop_dict[str(region_)] = drop_list
    print(datetime.datetime.now(), drop_list)

    r_values_xr, all_weights_xr = train_linear_model_remove(
        model_name='CMIP6', month_=month__, region_list=[region_key], 
        lag_list=np.arange(1,21), start_end_yr=[1941,2014], 
        n_epoch=2000, learn_rate=5e-4, extra_drop_=drop_list,
        white_noise_=False,
    )  

    CMIP6_r_vals.append(r_values_xr)
    CMIP6_weights.append(all_weights_xr)
        
region_r_vals = xr.concat((CMIP6_r_vals), dim='region')
region_r_vals['region'] = [1,2,3,4,5,6,11]

region_weights = xr.concat((CMIP6_weights), dim='region')
region_weights['region'] = [1,2,3,4,5,6,11]

r_values_attrs = {
    'Description': 'Pearson correlation coefficient for validation '\
        +'data from all second ensemble members from 42 CMIP6 Global Climate '\
        +f'Models, train on first members. The full list of GMCs is as '\
        +f'follows: {CMIP6_GCM_list}, for regions [1,2,3,4,5,6,11] with '\
        +f'months corresponding to {good_GCM_months}. The modes of '\
        +f'variability yielding lower predictive skill than a random variable '\
        +f'are dropped as follows: {drop_dict}. The full list of 15 '\
        +'variables is: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, NPI, NAM, '\
        +'NPO, PNA, NAO, SAM, TAS, as computed by the Climate Variability '\
        +'Diagnostics Package (CVDP) with 2 year lowpass filtered '\
        +'sea ice concentration. The linear model is trained at '\
        +f'different lag times of 1-20 years, the regions are defined '\
        +'by NSIDC MASIE-NH Version 1 (doi:10.7265/N5GT5K3K). '\
        +'Training uses an L1 loss function and Adam optimizer with 2000 '\
        +'epochs: nn.Sequential(nn.Linear(n_features,1,bias=False)), '\
        +'learning rate = 5e-4.',
    'Timestamp'  : str(datetime.datetime.utcnow().strftime(
        "%H:%M UTC %a %Y-%m-%d")),
    'Data source': f'CMIP6 global climate model historical simulations '\
        +'for historical sea ice concentration, with climate modes '\
        +'calculated by CVDP (doi:10.1002/2014EO490002)',
    'Analysis'   : 'https://github.com/chrisrwp/low-frequency-'\
        +'variability/blob/main/neural_network/Train_4_ML_Models.ipynb',
}

region_r_vals.attrs = r_values_attrs
region_r_vals.to_netcdf(
    '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
    +f'validation_r_values_linear_CMIP6_all_regions_best_months_'\
    +f'dropped_useless_vars.nc'
)


weights_attrs = r_values_attrs.copy()
weights_attrs['Description'] = 'Linear weights of the useful modes of '\
    +'variability as computed by the Climate Variability Diagnoistics'\
    +'Package (CVDP) with 2 year lowpass filtered sea ice concentration '\
    +f'for all regions [1,2,3,4,5,6,11] for months {good_GCM_months}. '\
    +f'The linear model is trained/validated on the first/second of all '\
    +f'members from the global climate models as follows {CMIP6_GCM_list}. '\
    +'Climate modes which do not yield a higher predictive skill than a '\
    +f'random variable are removed as follows {drop_dict}. The full list '\
    +'of variables is as follows: AMO, IPO, NINO34, PDO, AMM, ATN, IOD, '\
    +'NPI, NAM, NPO, PNA, NAO, SAM, TAS. The linear model is trained at '\
    +f'different lag times of 1-20 years and for region {region_key} as'\
    +'defined by NSIDC MASIE-NH Version 1 (doi:10.7265/N5GT5K3K). '\
    +'Training uses an L1 loss function and Adam optimizer with 2000 '\
    +'epochs: nn.Sequential(nn.Linear(n_features,1,bias=False)), '\
    +'learning rate = 5e-4.'

region_weights.attrs = weights_attrs
region_weights.to_netcdf(
    '/glade/work/cwpowell/low-frequency-variability/PyTorch_models/'\
    +f'weights_linear_CMIP6_all_regions_best_months_'\
    +f'dropped_useless_vars.nc'
)